<h1>Proyecto, Procesamiento de lenguaje natural aplicado al análisis de opinión política para las elecciones presidenciales en Colombia 2022<span class="tocSkip"></span></h1> 

>**Maestría en Analítica de Datos**  
>**Facultad de Ingeniería y Ciencias Básicas.**  
>**Universidad Central  2021 - III**  
>**Integrantes del trabajo:**  
>- Maria Alejandra Castillo Pabon
>- David Alejandro Ballesteros Díaz

# Librerias
Las librerias principales a instalar para el proceso de extracción son:
* **tweepy**: API proporcianada por Tweeter para extraer información de la red social, pero con restricciones de cantidad al ser gratuita.
* **selenium**: Bot para extracción de información de paginas HTML.

In [17]:
#!pip install tweepy
#!pip install selenium
#!pip install python-dotenv
from dotenv import load_dotenv
import os
load_dotenv()
import re
import csv
import pandas as pd
import glob
import numpy as np
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from msedge.selenium_tools import Edge, EdgeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import tweepy
import json
from datetime import date
from datetime import datetime

# Funciones personalizadas
A continuación se describen las funciones personalizadas para extracción de la información, dividas en tres grandes grupos:
* **Selenium BOT**: Funciones encaminadas para activación del BOT y extracción de la información de tweets publicados por los usuarios de Tweeter.
* **Tweepy**: Funciones encaminadas para la extracción de la información por usuario del perfil general y las usuarios que sigue.
* **complementos**: Funciones para guardar la información en formato CSV y modificar texto en campos numéricos.

## Selenium Bot

### Activar Bot
Función para abrir la ventana de EDGE en modo automático.  
Nota: El Bot esta configurado en navegador ya que presenta un mayor rendimiento, al momento de extracción de la información de una página HTML.

In [1]:
def controlador ():
    options = EdgeOptions()
    options.use_chromium = True
    driver = Edge(options=options)
    return(driver)

### Ir a una url
Función para excribir automaticamente dentro de la barra de direcciones la URL que se indique.

In [2]:
def ir_pagina (driver,url):
    driver.get(url)

### Tweets por usuario
Función para crear la URL e ir a la pagina HTML donde se encuentre la información de los tweets de un usuario por rango de fechas. 

In [3]:
def pagina_tweet (driver,usuario,fecha_inicio=0,fecha_fin=0):
    if fecha_inicio==0 and fecha_fin==0:
        url='https://twitter.com/'+usuario+'/with_replies'
    else:
        url='https://twitter.com/search?q=%23from%3A'+usuario+'%20since%3A'+fecha_inicio+'%20until%3A'+fecha_fin+'&src=typed_query&f=live'
    ir_pagina(driver,url)
    driver.execute_script("document.body.style.zoom='80%'")
    return ('conectado a la página '+url)

### Obtener tweet
Función para extraer por cada carta o grupo de sección de un HTML la información del tweet.

In [5]:
def obtener_tweet(card):
    try:
        username = (card.find_element_by_xpath('.//div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]').text).split('\n', 1)[0]
    except:
        username=""
    try:
        handle = (card.find_element_by_xpath('.//div[2]/div[2]/div[1]/div[1]/div[1]/div[1]/div[1]').text).split('\n', 1)[0]
    except:
        handle=""
    try:
        postdate = card.find_element_by_xpath('.//time').get_attribute('datetime')
    except:
        postdate=""
    try:
        text = card.find_element_by_xpath('.//div[2]/div[2]/div[2]/div[2]/div[1]').text
    except:
        text=1
    if text == 1:
        try:
            text = card.find_element_by_xpath('.//div[2]/div[2]/div[2]/div[1]').text
        except:
            text=""
    try:
        reply_cnt = card.find_element_by_xpath('.//div[@data-testid="reply"]').text
    except:
        reply_cnt=""
    try:
        retweet_cnt = card.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    except:
        retweet_cnt=""
    try:
        like_cnt = card.find_element_by_xpath('.//div[@data-testid="like"]').text 
    except:
        like_cnt=""
    try:
        #link= card.find_elements_by_css_selector('.css-4rbku5.css-18t94o4.css-901oao.r-14j79pv.r-1loqt21.r-1q142lx.r-37j5jr.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0')[0].get_attribute('href')#Male
        link= card.find_elements_by_css_selector('.css-4rbku5.css-18t94o4.css-901oao.r-9ilb82.r-1loqt21.r-1q142lx.r-37j5jr.r-a023e6.r-16dba41.r-rjixqe.r-bcqeeo.r-3s2u2q.r-qvutc0')[0].get_attribute('href')#David  
    except:
        link=""
    try:
        emoji_tags = card.find_elements_by_xpath('.//img[contains(@src, "emoji")]')
        emoji_list = []
        for tag in emoji_tags:
            filename = tag.get_attribute('src')
            try:
                emoji = chr(int(re.search(r'svg\/([a-z0-9]+)\.svg', filename).group(1), base=16))
            except :
                continue
            if emoji:
                emoji_list.append(emoji)
        emojis = ' '.join(emoji_list)
    except:
        emojis=''
    tweet = (username, handle, postdate, text, emojis, reply_cnt, retweet_cnt, like_cnt,link)
    return tweet

### Extracción Tweets
Función que toma cada elementos de la pagina HTML que contienen varios tweets, y a través de la función "obtener tweets", crea una lista con la información correspondiente de cada tweet. 

In [6]:
def inf_tweets (driver,fecha_fin = "01-01-2001"):
    data = []
    tweet_ids = set()
    last_position = driver.execute_script("return window.pageYOffset;")
    scrolling = True
    while scrolling:
        page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
        for card in page_cards[-15:]:
            tweet = obtener_tweet(card)
            if tweet is None:
                pass              
            else:
                if datetime.strptime("01-01-2020",'%d-%m-%Y')>datetime.strptime(fecha_fin,'%d-%m-%Y'):
                    tweet_id = ''.join(tweet)
                    if tweet_id not in tweet_ids:
                        tweet_ids.add(tweet_id)
                        data.append(tweet)                    
                else:
                    scrolling=False
        scroll_attempt = 0
        while True:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            sleep(2)
            curr_position = driver.execute_script("return window.pageYOffset;")
            if last_position == curr_position:
                scroll_attempt += 1

                if scroll_attempt >= 3:
                    scrolling = False
                    break
                else:
                    sleep(2)
            else:
                last_position = curr_position
                break
    print("fin extracción")
    return (data,datetime.strptime((tweet[2][0:10]), '%Y-%m-%d'))

### Guardar Tweets
Función para guardar en CSV los tweets contenidos en la lista.

In [7]:
def guardar_tweets (nombre_archivo,lista):
    with open(nombre_archivo, 'w', newline='', encoding="utf-8") as f:
        header = ['nombre', 'usuario', 'fecha', 'texto', 'emoticon', 'cantidad_comentarios', 'cantidad_likes', 'Cantidad_Retweets','link']
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(lista)
    return ('Se guardo el arhivo '+nombre_archivo+' con '+str(len(lista))+' registros')

## Tweepy
### Parámetros
Función para acceder, autenticarse y activar la API. 

In [11]:
consumer_key = os.getenv("consumer_key")
consumer_secret = os.getenv("consumer_secret")
access_token = os.getenv("access_token")
access_token_secret = os.getenv("access_token_secret")
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify=True)

### Perfil del usuario
Función para extracción de la información general del usuario de Tweeter.

In [19]:
def perfil_usuario(nombre):
    columnas = ['fecha_reporte','id', 'nombre', 'usuario','descripcion','cantidad_seguidores','cantidad_sigue','fecha_creacion']
    df_inf_general = pd.DataFrame(columns=columnas)
    data = api.get_user("@"+nombre)
    id_usuario=data._json ["id"]
    sigue=data._json ["friends_count"]
    df_inf_general=df_inf_general.append({'fecha_reporte':date.today(),
                                         'id':id_usuario,
                                         'nombre':data._json ["name"],
                                         'usuario':data._json ["screen_name"],
                                         'descripcion':data._json ["description"],
                                         'cantidad_seguidores':data._json ["followers_count"],
                                         'cantidad_sigue':sigue,
                                         'fecha_creacion':data._json ["created_at"]},ignore_index=True)
    return(df_inf_general)

### Usuarios que sigue un usuario
Función para extraer la información de los usuarios que sigue otro usuario de tweeter.

In [13]:
def sigue_a(numero_id,cantidad):
    columnas = ['id', 'nombre', 'usuario','descripcion','cantidad_seguidores','cantidad_sigue','fecha_creacion']
    df_sigue = pd.DataFrame(columns=columnas)
    for user in tweepy.Cursor(api.friends, id=numero_id).items(cantidad):
        df_sigue=df_sigue.append({'id':user._json ["id"],
                                  'nombre':user._json ["name"],
                                  'usuario':user._json ["screen_name"],
                                  'descripcion':user._json ["description"],
                                  'cantidad_seguidores':user._json ["followers_count"],
                                  'cantidad_sigue':user._json ["friends_count"],
                                  'fecha_creacion':user._json ["created_at"]},ignore_index=True)
    return(df_sigue)

## Complementos

### Guardar Dataframe - CSV
Función para guardar los datos de un DataFrame en formato CSV.

In [14]:
def guardar_csv (df,nombre_archivo):
    df.to_csv(nombre_archivo+'.csv', index=False,encoding='utf-8')
    return('El archivo '+nombre_archivo+'.csv fue guardado exitosamente')

### Texto de miles
Función para cambiar el valor de miles (español o ingles) a número.

In [147]:
def valor_k(x):
    x=str(x)
    if x.find(".")>0 or x.find(",")>0:
        x=int(re.sub('[.,k,K,\s,a-zA-Z]','',x))*100
    elif x =='nan':
        x=0
    elif x.find("K") >0 or x.find("mil") >0:
        x=int(re.sub('[k,K\s,a-zA-Z]','',x))*1000
    else:
        x= int(x)
    return(x)

# Proceso de Extracción

## Lista de usarios
Se crea una lista con los usuarios de tweeter de los precandidatos políticos a extraer la información.

In [20]:
usuarios=("OIZuluaga","MariaFdaCabal","ingrodolfohdez","petrogustavo","sergio_fajardo")

## Periodos de Extracción
Se crea un diccionario con los rangos requeridos para extraer los tweets de los usuarios.  
Nota: se requieren estos rangos debido a la limitante de memoria que puede tener una pagina html al cargar demasiada información de tweets en esta.

In [101]:
periodos={
    'rango1': ['2021-08-28','2021-08-29'],
    'rango2': ['2021-08-24','2021-08-27']
         }

## Perfil de los usuarios en Twitter
Se itera por cada uno de los usuarios para extraer y guardar su perfil en formato CSV.

In [179]:
df=pd.DataFrame()
for usuario in usuarios:
    df0=perfil_usuario(usuario)
    df=pd.concat([df0, df])
guardar_csv(df,"Inf_General_Candidatos_Twitter")

'El archivo Inf_General_Candidatos_Twitter.csv fue guardado exitosamente'

## Usuarios a los que se siguen en Twitter
Se itera por cada uno de los usuarios para extaer y guardar los usuarios que ellos siguen a través de twitter.

In [174]:
for id_usuario in df.id[0]:
    df_1 = df[df['id'] == id_usuario]
    usuario=df_1.usuario[0]
    cantidad_sigue=df_1.cantidad_sigue[0]
    nombre_archivo = usuario+"_entorno"
    df=sigue_a(id_usuario,cantidad_sigue)
    print(guardar_csv(df,nombre_archivo))

El archivo sergio_fajardo_entorno.csv fue guardado exitosamente
El archivo petrogustavo_entorno.csv fue guardado exitosamente
El archivo ingrodolfohdez_entorno.csv fue guardado exitosamente
El archivo MariaFdaCabal_entorno.csv fue guardado exitosamente
El archivo OIZuluaga_entorno.csv fue guardado exitosamente


## Tweets de cada precandidato
Se realiza la activación del bot y se itera por cada usuario y periodo, para ir a la página de sus tweets y extraer y guaradar cada uno de ellos.

In [246]:
driver=controlador()
sleep(5)
for rango in periodos:
    fecha_inicio = periodos[rango][0]
    fecha_fin = periodos[rango][1]
    for usuario in usuarios:
        pagina_tweet(driver,usuario,fecha_inicio,fecha_fin)
        tweets=inf_tweets(driver)
        nombre_archivo=[usuario+"_"+fecha_inicio+"_"+fecha_fin+'.csv']
        guardar_tweets (nombre_archivo[0],tweets[0])

fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción


## Top 5 links mas comentados de cada precandidato
Se crea un proceso para extracción de las respuestas sobre los tweets mas comentados (top 5) por cada usuario.

### Identificación de los links
Se itera por cada usuario para identificar los 5 tweets que mas comentarios generaron.

In [135]:
ruta='C:\\Users\David\Python\PNL\Tweets'
for usuario in usuarios:
    df=pd.DataFrame()
    try: 
        lista=[file_name for file_name in glob.glob(ruta+'/'+'*.csv') if file_name.find(usuario) > 0 ]
        for archivo in lista:
            df0= pd.read_csv(archivo,index_col=False)
            df=pd.concat([df0, df])    
        df['cantidad_comentarios']=df['cantidad_comentarios'].apply(valor_k)     
        links=[link for link in df.sort_values('cantidad_comentarios',ascending=False).head(5).link]
        locals()["links_"+usuario] = links              
    except:
        pass

In [136]:
print(links_sergio_fajardo,'\n')
print(links_OIZuluaga,'\n')
print(links_MariaFdaCabal,'\n')
print(links_ingrodolfohdez,'\n')
print(links_petrogustavo,'\n')

['https://twitter.com/sergio_fajardo/status/1297702268020625408', 'https://twitter.com/sergio_fajardo/status/1283217336816865280', 'https://twitter.com/sergio_fajardo/status/1333797916486483968', 'https://twitter.com/sergio_fajardo/status/1285768190656688129', 'https://twitter.com/sergio_fajardo/status/1402682869131223049'] 

['https://twitter.com/mluciaramirez/status/1271226549426143233', 'https://twitter.com/AlvaroUribeVel/status/1314942928998084610', 'https://twitter.com/jeronimoauribem/status/1292167893153652739', 'https://twitter.com/IvanDuque/status/1390733669564887043', 'https://twitter.com/OIZuluaga/status/1426974730649747460'] 

['https://twitter.com/MariaFdaCabal/status/1397007956022022146', 'https://twitter.com/MariaFdaCabal/status/1398330107975061507', 'https://twitter.com/MariaFdaCabal/status/1346614913406558208', 'https://twitter.com/MariaFdaCabal/status/1379837106755936258', 'https://twitter.com/MariaFdaCabal/status/1395365369125261313'] 

['https://twitter.com/ingrodolf

### Extracción
A través de cada uno de los links mas comentados se ingresa a estos y se extrae su información.

In [146]:
usuarios=("OIZuluaga","MariaFdaCabal","ingrodolfohdez","petrogustavo","sergio_fajardo")
driver=controlador()
sleep(5)
for usuario in usuarios:
    links=locals()["links_"+usuario]
    i=0
    for link in links:
        i=i+1
        driver.get(link)
        driver.execute_script("document.body.style.zoom='50%'")
        tweets=inf_tweets(driver)
        guardar_tweets (usuario+"_link"+str(i)+".csv",tweets[0])

fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
fin extracción
